In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../..")

In [ ]:
import os
import analytiq_data as ad
import asyncio

In [ ]:
ad.common.setup()

In [ ]:
# Set the MONGODB_URI environment variable
os.environ["MONGODB_URI"] = "mongodb://localhost:27017"

In [ ]:
analytiq_client = ad.common.get_analytiq_client(env="test")

llm_key = await ad.llm.get_llm_key(analytiq_client, llm_provider="mistral")
print(llm_key)

In [ ]:
org_id = "6795345439604beca2b2808d"
prompt_id, version = await ad.common.get_prompt_id(analytiq_client, "cv", org_id)
prompt_id, version

In [ ]:
prompt_revid = await ad.common.get_prompt_revid(analytiq_client, prompt_id, version)
prompt_revid

In [ ]:
prompt_revid = "68926f54b175d0e0ff51b11c"

In [ ]:
document_id = "687c5bd2eecf1be4639ea6d0"

In [ ]:
await ad.llm.get_llm_model(analytiq_client, prompt_revid)

In [ ]:
# Clear the AWS credentials in the env to avoid using them
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_REGION_NAME"] = ""

In [ ]:
llm_result = await ad.llm.run_llm(analytiq_client,
                                  document_id=document_id,
                                  prompt_revid=prompt_revid,
                                  #llm_model="bedrock/anthropic.claude-3-7-sonnet-20250219-v1:0",
                                  #llm_model="anthropic.claude-3-5-sonnet-20240620-v1:0",
                                  #llm_model="groq/deepseek-r1-distill-llama-70b",
                                  #llm_model="gemini/gemini-2.0-flash",
                                  #llm_model="gemini/gemini-2.5-flash-preview-05-20",
                                  #llm_model="gemini/gemini-2.5-flash-preview-tts",
                                  #llm_model="azure_ai/deepseek-v3",
                                  llm_model="gpt-4.5-preview",
                                  #llm_model="gpt-4o-2024-08-06",
                                  #llm_model="claude-3-5-sonnet",
                                  #llm_model="mistral/mistral-large-latest",
                                  #llm_model="mistral/open-mixtral-8x22b",
                                  force=True)
llm_result


In [ ]:
llm_result_id = await ad.llm.save_llm_result(analytiq_client,
                                             document_id=document_id,
                                             prompt_revid=prompt_revid,
                                             llm_result=llm_result)
llm_result_id

In [ ]:
llm_result = await ad.llm.get_llm_result(analytiq_client,
                                         document_id=document_id,
                                         prompt_revid=prompt_revi)
llm_result


In [ ]:
await ad.llm.delete_llm_result(analytiq_client,
                                document_id=document_id,
                                prompt_revid=prompt_revid)


In [ ]:
await ad.llm.run_llm_for_prompt_revids(analytiq_client,
                                        document_id=document_id,
                                        prompt_revids=[prompt_revid],
                                        model="groq/deepseek-r1-distill-llama-70b")


In [ ]:
from litellm.utils import supports_response_schema

supports_response_schema(model="groq/deepseek-r1-distill-llama-70b")

In [ ]:
await ad.llm.list_llm_providers(analytiq_client)

In [ ]:
ad.llm.is_chat_model("mistral/mistral-large-latest")

In [ ]:
ad.llm.get_supported_models()

In [ ]:
ad.llm.get_available_models()

In [ ]:
for model in ad.llm.get_supported_models():
    if not ad.llm.is_supported_model(model):
        raise Exception(f"Model {model} is not supported")

In [ ]:
litellm.models_by_provider.keys()

In [ ]:
await ad.llm.providers.setup_llm_providers(analytiq_client)

In [ ]:
import os
from litellm import completion

aws_config = await ad.aws.get_aws_config(analytiq_client)
aws_access_key_id = aws_config["aws_access_key_id"]
aws_secret_access_key = aws_config["aws_secret_access_key"]
aws_region_name = "us-east-1"

response = completion(
  #provider="bedrock",
  #model="bedrock/anthropic.claude-sonnet-4-20250514-v1:0",
  #model="bedrock/us.anthropic.claude-sonnet-4-20250514-v1:0",
  #model="bedrock/arn:aws:bedrock:us-east-1:890742589311:inference-profile/claude-sonnet-4-profile",
  #model="bedrock/anthropic.claude-3-7-sonnet-20250219-v1:0",
  #model="anthropic.claude-3-5-sonnet-20240620-v1:0",
  #model="gemini/gemini-2.0-flash",
  #model="gemini/gemini-2.5-flash-preview-05-20",
  model="gemini/gemini-2.5-pro-preview-06-05",
  messages=[{ "content": "Hello, how are you?","role": "user"}],
  #response_format = {"type": "json_object"},
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key,
  aws_region_name=aws_region_name,
)

response

In [ ]:
import litellm
from litellm import CustomLLM, completion, get_llm_provider


class MyCustomLLM(CustomLLM):
    def completion(self, *args, **kwargs) -> litellm.ModelResponse:
        return litellm.completion(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Hello world"}],
            mock_response="Hi!",
        )  # type: ignore

In [ ]:
my_custom_llm = MyCustomLLM()

litellm.custom_provider_map = [ # ðŸ‘ˆ KEY STEP - REGISTER HANDLER
        {"provider": "my-custom-llm", "custom_handler": my_custom_llm}
    ]

In [ ]:
resp = completion(
        model="my-custom-llm/my-fake-model",
        messages=[{"role": "user", "content": "Hello world!"}],
    )

assert resp.choices[0].message.content == "Hi!"

In [ ]:
resp